# Вебинар 4. Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

---

Импортируем библиотеки и рукописные модули из src - ```metrics```, ```utils``` и ```ecommenders```:

In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

from src.metrics import precision_at_k, recall_at_k 
from src.recommenders import MainRecommender
from src.utils import prefilter_items, get_result_table

import warnings
warnings.filterwarnings('ignore') 

Качаем файлы:

In [2]:
data = pd.read_csv('../webinar_4/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'}, inplace=True)

In [4]:
item_features = pd.read_csv('../webinar_4/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


Делим датасет на тренировочную и тестовую выборки, оставляя на тестовую последние 3 недели:

In [3]:
data_train = data[data['week_no'] < data['week_no'].max() - 3]
data_test = data[data['week_no'] >= data['week_no'].max() - 3]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


Собираем items из тестовой выборки, сохраняем их в датафрейм result (функция для этого лежит в src.metrics):

In [5]:
result = get_result_table(data_test)
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


С помощью функции prefilter_items из src.metrics снижаем количество элементов:

In [6]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print(f'Количество элементов уменьшено с {n_items_before} до {n_items_after}')

Количество элементов уменьшено с 86865 до 5001


С помощью метода prepare_matrix из самописного класса MainRecommender достаём разреженную матрицу user_item_matrix (просто чтоб убедиться в том, что она формируется нормально и её вот так вот можно доставать):

In [7]:
main_data_train = MainRecommender(data_train)
user_item_matrix = main_data_train.user_item_matrix
user_item_matrix.head(3)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


И словари (с той же целью, просто убедиться в том, что всё работает):

In [8]:
id_to_itemid, id_to_userid, itemid_to_id, userid_to_id =\
    main_data_train.id_to_itemid, main_data_train.id_to_userid, main_data_train.itemid_to_id, main_data_train.userid_to_id

Получим рекомендацию для одного пользователя, допустим это пользователь №5. Для этого нужно просто указать аргумент num_user:

In [9]:
mr_5 = MainRecommender(data_train, als_recommend=True, num_user=5)

  0%|          | 0/15 [00:00<?, ?it/s]

Посчитаем для полученных рекомендаций этому пользователю precision_at_k:

In [10]:
mr_5_precision = precision_at_k(mr_5.recommender, result.actual.loc[3], k=5)
mr_5_precision

0.2

А вот так можно получить словарь с рекомендациями для всех пользователей (это долго):

In [11]:
%%time
mr_all = MainRecommender(data_train, als_recommend=True)
mr_all.generate_recommendations()

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 12min


Из полученных рекомендация для всех посмотрим рекомендации для кого-то одного, например, для пользователя №123:

In [12]:
recs_123 = mr_all.recommendations[123]

In [13]:
recs_123

[5569230, 831628, 1070702, 12810393, 944534]

$\Rightarrow$ - из каждого самописного модуля мы что-то использовали и тем самым убедились в том, что все импорты работают и работают корректно.

-----
